In [123]:
import torch 

In [124]:
with open ('dataset/train.csv', 'r', encoding='utf-8') as f:
    text = f.read()

In [125]:
print("char_length:", len(text))
print(text[:1000])

char_length: 1003862
text
"First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, n

In [126]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
66


In [127]:


stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("hi there"))
print(decode(encode("hi there")))


[47, 48, 1, 59, 47, 44, 57, 44]
hi there


In [128]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1003862]) torch.int64
tensor([59, 44, 63, 59,  0,  3, 19, 48, 57, 58, 59,  1, 16, 48, 59, 48, 65, 44,
        53, 11,  0, 15, 44, 45, 54, 57, 44,  1, 62, 44,  1, 55, 57, 54, 42, 44,
        44, 43,  1, 40, 53, 64,  1, 45, 60, 57, 59, 47, 44, 57,  7,  1, 47, 44,
        40, 57,  1, 52, 44,  1, 58, 55, 44, 40, 50,  9,  0,  0, 14, 51, 51, 11,
         0, 32, 55, 44, 40, 50,  7,  1, 58, 55, 44, 40, 50,  9,  0,  0, 19, 48,
        57, 58, 59,  1, 16, 48, 59, 48, 65, 44, 53, 11,  0, 38, 54, 60,  1, 40,
        57, 44,  1, 40, 51, 51,  1, 57, 44, 58, 54, 51, 61, 44, 43,  1, 57, 40,
        59, 47, 44, 57,  1, 59, 54,  1, 43, 48, 44,  1, 59, 47, 40, 53,  1, 59,
        54,  1, 45, 40, 52, 48, 58, 47, 13,  0,  0, 14, 51, 51, 11,  0, 31, 44,
        58, 54, 51, 61, 44, 43,  9,  1, 57, 44, 58, 54, 51, 61, 44, 43,  9,  0,
         0, 19, 48, 57, 58, 59,  1, 16, 48, 59, 48, 65, 44, 53, 11,  0, 19, 48,
        57, 58, 59,  7,  1, 64, 54, 60,  1, 50, 53, 54, 62,  1, 16, 40, 48, 60,
      

In [129]:
n = int(0.9*len(data))
train_data = data [:n]
val_data = data[n:]

In [130]:


block_size = 8
train_data[:block_size+1]

tensor([59, 44, 63, 59,  0,  3, 19, 48, 57])

In [131]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range (block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")
    

when input is tensor([59]) the target: 44
when input is tensor([59, 44]) the target: 63
when input is tensor([59, 44, 63]) the target: 59
when input is tensor([59, 44, 63, 59]) the target: 0
when input is tensor([59, 44, 63, 59,  0]) the target: 3
when input is tensor([59, 44, 63, 59,  0,  3]) the target: 19
when input is tensor([59, 44, 63, 59,  0,  3, 19]) the target: 48
when input is tensor([59, 44, 63, 59,  0,  3, 19, 48]) the target: 57


In [132]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8 

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix ])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y 

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('---')

for b in range(batch_size):
        for t in range(block_size):
              context = xb[b, :t+1]
              target = yb[b,t]
              print(f"when input is {context.tolist()} the target: {target}")
              


inputs:
torch.Size([4, 8])
tensor([[ 1, 58, 59, 40, 64,  1, 59, 47],
        [40, 61, 44,  1, 59, 48, 51, 51],
        [ 7,  1, 58, 48, 59,  1, 52, 64],
        [43, 40, 51,  1, 45, 51, 54, 62]])
targets:
torch.Size([4, 8])
tensor([[58, 59, 40, 64,  1, 59, 47, 44],
        [61, 44,  1, 59, 48, 51, 51,  1],
        [ 1, 58, 48, 59,  1, 52, 64,  1],
        [40, 51,  1, 45, 51, 54, 62, 44]])
---
when input is [1] the target: 58
when input is [1, 58] the target: 59
when input is [1, 58, 59] the target: 40
when input is [1, 58, 59, 40] the target: 64
when input is [1, 58, 59, 40, 64] the target: 1
when input is [1, 58, 59, 40, 64, 1] the target: 59
when input is [1, 58, 59, 40, 64, 1, 59] the target: 47
when input is [1, 58, 59, 40, 64, 1, 59, 47] the target: 44
when input is [40] the target: 61
when input is [40, 61] the target: 44
when input is [40, 61, 44] the target: 1
when input is [40, 61, 44, 1] the target: 59
when input is [40, 61, 44, 1, 59] the target: 48
when input is [40, 61, 4

In [133]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):

        for _ in range(max_new_tokens):
        
            logits, loss = self(idx)
     
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1) 
          
            idx_next = torch.multinomial(probs, num_samples=1)
            
            idx = torch.cat((idx, idx_next), dim=1) 
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 66])
tensor(4.5633, grad_fn=<NllLossBackward0>)

w&&S-d:CyEXkWwz-ZamuSHW
&ECDM3llkn "cZQjw-UYmLLO$ZnsHs
wd"ygtuMaczY-F;EIEVjuAF"Uk?PsSKRYBoEIQjZZKevc


In [134]:

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [135]:
batch_size = 32
for steps in range(100000): 


    xb, yb = get_batch('train')


    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.491783380508423


In [136]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Gindeghed thasores minthe; les qul mer, tindory fiene wima m, r LI s han?
Whe is, I thesar m Fis:
TI:
Thy, oon woninjun iss mave
Thy be,

A:
MO that her is wishaler he t Romerd y inoresor s e thuers g ly d a aline ds;
Kithed: ke whomime GERI ero ucessskste dat resuld hengr bawer omappl nos; thindeis ye pland:
ARLO:
Whirg swatu te hengsouck thay, himavete; cy Hedaway,
Asovieers, opor tthe.
RK:
FOLongeemy GAsofohwferakes CI If I blouppse msore sho CEl t moy cean; meree s and. wis
I:
Pavef s tthe t


In [137]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

torch.manual_seed(1337)
B,T,C = 4,8,2 
x = torch.randn(B,T,C)
x.shape

xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [138]:

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [139]:

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [140]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])